In [86]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import researchpy as rp
import pymer4
from pymer4.models import Lmer
from pymer4.models import Lm
from pymer4.utils import get_resource_path
import os
from sklearn.preprocessing import OneHotEncoder

In [87]:
# Load your dataset
df = pd.read_csv(os.path.join(get_resource_path(), r'C:\Users\alvarocairo\Insurance.csv'))
# Display the data
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,1,southwest,16884.92
1,18,male,33.8,1,0,southeast,1725.55
2,28,male,33.0,3,0,southeast,4449.46
3,33,male,22.7,0,0,northwest,21984.47
4,32,male,28.9,0,0,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,0,northwest,10600.55
1334,18,female,31.9,0,0,northeast,2205.98
1335,18,female,36.9,0,0,southeast,1629.83
1336,21,female,25.8,0,0,southwest,2007.95


In [88]:
# Check what is in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 73.3+ KB


In [89]:
rp.codebook(df) # get a better understanding of the data

Variable: age    Data Type: int64 

 Number of Obs.: 1338 
 Number of missing obs.: 0 
 Percent missing: 0.0 
 Number of unique values: 47 

 Range: [18, 64] 
 Mean: 39.21 
 Standard Deviation: 14.05 
 Mode: 18 
 10th Percentile: 19.0 
 25th Percentile: 27.0 
 50th Percentile: 39.0 
 75th Percentile: 51.0 
 90th Percentile: 59.0 





Variable: sex    Data Type: object 

 Number of Obs.: 1338 
 Number of missing obs.: 0 
 Percent missing: 0.0 
 Number of unique values: 2 

 Data Values and Counts: 
 
 Values  Frequency
female        662
  male        676




Variable: bmi    Data Type: float64 

 Number of Obs.: 1338 
 Number of missing obs.: 0 
 Percent missing: 0.0 
 Number of unique values: 275 

 Range: [16.0, 53.1] 
 Mean: 30.67 
 Standard Deviation: 6.1 
 Mode: 27.6 
 10th Percentile: 23.0 
 25th Percentile: 26.3 
 50th Percentile: 30.4 
 75th Percentile: 34.7 
 90th Percentile: 38.629999999999995 





Variable: children    Data Type: int64 

 Number of Obs.: 1338 
 Number of mi

In [90]:
IV1 = df[['age', 'bmi']]  # Independent Variables 1: age and bmi
IV3 = df[['sex', 'children', 'smoker', 'region']]  # Independent Variables 2: sex, children, smoker, region
DV = df['expenses'] # Dependent Variable: expenses

In [91]:
# Replace values in the 'children' column
df['children'] = df['children'].replace({0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1})

In [92]:
bmi = {
    '0-18.5': (0,18.5), #underweight range
    '18.51-25': (18.51,25), #normal weight range
    '25.01-29.99': (25.01,29.99), #overweight range 
    '30.0 - 34.99': (30,34.99), #obesity range
    '35.0 and above':(35, float('inf')) #severe obesity range
}

In [93]:
# Map 'male' to 0 and 'female' to 1
df['sex'] = df['sex'].replace({'male': 0, 'female': 1})

# Display the resulting DataFrame
df

,age,sex,bmi,children,smoker,region,expenses
0,19,1,27.9,0,1,southwest,16884.92
1,18,0,33.8,1,0,southeast,1725.55
2,28,0,33.0,1,0,southeast,4449.46
3,33,0,22.7,0,0,northwest,21984.47
4,32,0,28.9,0,0,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,0,31.0,1,0,northwest,10600.55
1334,18,1,31.9,0,0,northeast,2205.98
1335,18,1,36.9,0,0,southeast,1629.83
1336,21,1,25.8,0,0,southwest,2007.95


In [102]:
# Define your multilevel linear regression model with 'region' as the grouping variable
model = Lmer('expenses ~ age + sex + children + smoker + bmi +(age+sex+children+smoker+bmi|region)', data=df)

# Fit the model
results = model.fit()

# Get and print the model summary
print(results)

boundary (singular) fit: see help('isSingular') 

Linear mixed model fit by REML [’lmerMod’]
Formula: expenses~age+sex+children+smoker+bmi+(age+sex+children+smoker+bmi|region)

Family: gaussian	 Inference: parametric

Number of observations: 1338	 Groups: {'region': 4.0}

Log-likelihood: -13501.793 	 AIC: 27059.587

Random effects:

                 Name           Var       Std
region    (Intercept)  2.143000e+00     1.464
region            age  5.105700e+01     7.145
region            sex  1.782133e+04   133.497
region       children  2.258548e+05   475.242
region         smoker  1.168627e+07  3418.519
region            bmi  1.459316e+03    38.201
Residual               3.572998e+07  5977.456

                IV1       IV2   Corr
region  (Intercept)       age  1.000
region  (Intercept)       sex  0.508
region  (Intercept)  children -0.262
region  (Intercept)    smoker -0.005
region  (Intercept)       bmi -0.240
region          age       sex  0.508
region          age  children -0.262


In [95]:
model.fixef

,(Intercept),age,sex,children,smoker,bmi
northeast,-12730.499886,252.677745,91.191003,1460.301270,20884.671228,374.132150
northwest,-12729.866110,255.771551,137.830020,1290.611929,21915.440240,359.174487
southeast,-12729.491902,257.598263,201.553907,709.019073,26496.988954,309.417810
southwest,-12729.162803,259.204776,231.266308,731.673618,25321.828512,320.400427


In [96]:
model.ranef

,X.Intercept.,age,sex,children,smoker,bmi
northeast,-0.744711,-3.635338,-74.269307,412.399798,-2770.061005,33.350932
northwest,-0.110935,-0.541533,-27.630289,242.710457,-1739.291994,18.393269
southeast,0.263273,1.285179,36.093597,-338.882400,2842.256721,-31.363409
southwest,0.592372,2.891692,65.805999,-316.227855,1667.096279,-20.380791


In [97]:
print(model.coefs)

                 Estimate        2.5_ci       97.5_ci           SE  \
(Intercept) -12729.755175 -14610.978754 -10848.531597   959.825585   
age            256.313084    232.223076    280.403091    12.291046   
sex            165.460310   -491.588549    822.509168   335.235169   
children      1047.901472    249.251974   1846.550971   407.481721   
smoker       23654.732234  20210.143322  27099.321145  1757.475616   
bmi            340.781218    275.410514    406.151923    33.353013   

                      DF     T-stat         P-val  Sig  
(Intercept)  1031.428480 -13.262571  3.499023e-37  ***  
age             6.235446  20.853643  5.281008e-07  ***  
sex            31.766310   0.493565  6.250088e-01       
children        5.383384   2.571653  4.661662e-02    *  
smoker          3.291460  13.459494  5.440114e-04  ***  
bmi            13.684992  10.217404  8.809577e-08  ***  


In [101]:
# Define the formula for your multilevel model
formula = "expenses ~ age + sex + children + smoker + C(region) + age:region + sex:region + children:region + smoker:region"

# Fit the multilevel model
model = sm.MixedLM.from_formula(formula, data=df, groups=df["region"]).fit()

# Print the model summary
print(model.summary())


                       Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         expenses     
No. Observations:          1338            Method:                     REML         
No. Groups:                4               Scale:                      39899745.9409
Min. group size:           324             Log-Likelihood:             -13463.2984  
Max. group size:           364             Converged:                  Yes          
Mean group size:           334.5                                                    
------------------------------------------------------------------------------------
                                Coef.     Std.Err.   z    P>|z|   [0.025     0.975] 
------------------------------------------------------------------------------------
Intercept                       -2115.154 6422.996 -0.329 0.742 -14703.995 10473.688
C(region)[T.northwest]           -663.554 9085.316 -0.073 0.942 -18470.446 17143.337
C(re

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
